In [26]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import mediapipe as mp
import cv2
import numpy as np

In [28]:
def calculate_angle(a, b, c):
    """
    Calculate the angle between three points using the cosine rule
    """
    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])

    radians = np.arccos(np.dot(b - a, c - b) / (np.linalg.norm(b - a) * np.linalg.norm(c - b)))
    angle = np.degrees(radians)

    return angle

In [29]:
def count_shoulder_presses():
    """
    Count the number of shoulder presses performed in real-time using Mediapipe
    """
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)
    card_width, card_height = 180, 100
    card_x, card_y = 10, 10
    card_color = (255, 153, 13)

    cap = cv2.VideoCapture(0)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.7) as pose:
        press_count = 0
        is_pressing = False
        prev_angle = None
        prev_angle1 = None
        threshold=110

        while True:
            ret, image = cap.read()

            image = cv2.flip(image, 1)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            results = pose.process(image_rgb)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            if results.pose_landmarks is not None:
                # left 
                left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
                left_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
                left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
                
                # right
                right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
                right_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
                right_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]

                angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                # cv2.putText(image, f"Right Angle: {angle:.2f} degrees", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                angle1 = calculate_angle(right_shoulder, right_elbow, right_wrist)
                # cv2.putText(image, f"Right Angle: {angle1:.2f} degrees", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

                if prev_angle and prev_angle1 is not None:
                    delta = abs(angle - prev_angle)
                    delta1 = abs(angle1 - prev_angle1)
                    # cv2.putText(image, f"Right Angle: {delta1:.2f} degrees", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                    if not is_pressing and delta >8 and delta1 > 8:
                        is_pressing = True
                    elif is_pressing and delta < 5 and delta < 5:
                        is_pressing = False

                prev_angle = angle
                prev_angle1 = angle1

                if is_pressing and angle < threshold and angle1 < threshold:
                    press_count += 1
                    is_pressing = False

            cv2.rectangle(image, (card_x, card_y), (card_x + card_width, card_y + card_height), card_color, -1)

            # Display the text inside the card
            text_x, text_y = card_x + 10, card_y + 30
            text_line1 = '  Shoulder  '
            text_line3 = '   Press  '
            text_line2 = f'  Count: {press_count} '
            line_spacing = 30
            font_scale = 0.7
            font_color = (255, 255, 255)

            cv2.putText(image, text_line1, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX, font_scale, font_color, 2, cv2.LINE_AA)
            cv2.putText(image, text_line3, (text_x, text_y+ line_spacing), cv2.FONT_HERSHEY_COMPLEX, font_scale, font_color, 2, cv2.LINE_AA)
            cv2.putText(image, text_line2, (text_x, text_y + line_spacing+ line_spacing), cv2.FONT_HERSHEY_COMPLEX, font_scale, font_color, 2, cv2.LINE_AA)
            
            cv2.imshow('Shoulder Press counter', image)

            if cv2.waitKey(1) == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

    return press_count

In [30]:
count =count_shoulder_presses()